#  Machine Learning y Modelos Predictivos

## Objetivo
Este notebook desarrolla **modelos de machine learning avanzados** para predecir y optimizar las operaciones del call center, incluyendo predicción de abandonos, clasificación de llamadas y optimización de recursos.

## Contenido del Análisis
1. **Preparación de Datos para ML**
2. **Predicción de Abandonos (Clasificación)**
3. **Predicción de Tiempo de Servicio (Regresión)**
4. **Clasificación de Tipos de Llamada**
5. **Optimización de Asignación de Recursos**
6. **Detección de Anomalías**
7. **Modelos de Pronóstico de Demanda**
8. **Evaluación y Comparación de Modelos**
9. **Implementación y Deployment**

---

In [2]:
# Importamos las librerías para machine learning
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime, timedelta
import os
import json
import pickle
from scipy import stats
import sys

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve,
    mean_squared_error, mean_absolute_error, r2_score
)
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif

# Configuraciones
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Configuración de plotly
import plotly.io as pio
pio.renderers.default = "notebook"

sys.path.append('../02_src')
from feature_engineering import FeatureEngineer

print(" Librerías de Machine Learning importadas exitosamente")
print(f" Pandas versión: {pd.__version__}")
print(f" Scikit-learn disponible")

# Configurar semilla para reproducibilidad
np.random.seed(42)

 Librerías de Machine Learning importadas exitosamente
 Pandas versión: 2.2.3
 Scikit-learn disponible


## 1. Preparación de Datos para Machine Learning

In [3]:
# Cargamos los datos limpios
print(" Cargando datos para machine learning...")
df = pd.read_parquet('../00_data/processed/call_center_clean.parquet')

print(f" Datos cargados: {df.shape[0]:,} filas x {df.shape[1]} columnas")

# Preparamos las características temporales avanzadas
df['date'] = pd.to_datetime(df['date'])
df['vru_entry_dt'] = pd.to_datetime(df['vru_entry'])
df['hour'] = df['vru_entry_dt'].dt.hour
df['minute'] = df['vru_entry_dt'].dt.minute
df['day_of_week'] = df['date'].dt.dayofweek
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter
df['week_of_year'] = df['date'].dt.isocalendar().week
df['is_weekend'] = (df['date'].dt.weekday >= 5).astype(int)
df['is_monday'] = (df['date'].dt.weekday == 0).astype(int)
df['is_friday'] = (df['date'].dt.weekday == 4).astype(int)

# Características de la carga de trabajo
hourly_load = df.groupby(['date', 'hour'])['call_id'].count().reset_index()
hourly_load.columns = ['date', 'hour', 'hourly_calls']
df = df.merge(hourly_load, on=['date', 'hour'], how='left')

# Características de cliente (para clientes conocidos)
customer_history = df[df['customer_id'].notna()].groupby('customer_id').agg({
    'call_id': 'count',
    'outcome': lambda x: (x == 'HANG').sum(),
    'ser_time': 'mean'
}).round(2)

customer_history.columns = ['customer_call_count', 'customer_hang_count', 'customer_avg_service_time']
customer_history['customer_hang_rate'] = (customer_history['customer_hang_count'] / customer_history['customer_call_count'] * 100).round(2)

df = df.merge(customer_history, left_on='customer_id', right_index=True, how='left')

# Rellenamos con NaN para clientes nuevos/desconocidos
df['customer_call_count'] = df['customer_call_count'].fillna(1)
df['customer_hang_count'] = df['customer_hang_count'].fillna(0)
df['customer_avg_service_time'] = df['customer_avg_service_time'].fillna(df['ser_time'].median())
df['customer_hang_rate'] = df['customer_hang_rate'].fillna(0)

# Creamos las características de eficiencia
df['efficiency_ratio'] = df['ser_time'] / (df['ser_time'] + df['q_time'] + df['vru_time'] + 1)
df['total_time'] = df['ser_time'] + df['q_time'] + df['vru_time']
df['wait_time_ratio'] = (df['q_time'] + df['vru_time']) / (df['total_time'] + 1)

# Codificamos las variables categóricas
le_type = LabelEncoder()
le_vru_line = LabelEncoder()
le_server = LabelEncoder()

df['type_encoded'] = le_type.fit_transform(df['type'])
df['vru_line_encoded'] = le_vru_line.fit_transform(df['vru.line'])
df['server_encoded'] = le_server.fit_transform(df['server'])

print("️ Características de ML preparadas")
print(f" Total de características: {df.shape[1]}")

# Mostramos las estadísticas de outcomes
outcome_counts = df['outcome'].value_counts()
print("\n Distribución de outcomes:")
for outcome, count in outcome_counts.items():
    pct = count / len(df) * 100
    print(f"  {outcome}: {count:,} ({pct:.2f}%)")

 Cargando datos para machine learning...
 Datos cargados: 444,448 filas x 18 columnas
️ Características de ML preparadas
 Total de características: 39

 Distribución de outcomes:
  AGENT: 353,143 (79.46%)
  HANG: 87,706 (19.73%)
  PHANTOM: 3,599 (0.81%)


## 2. Modelo de Predicción de Abandonos (Clasificación)

In [16]:
# Instalamos las librerías necesarias
%pip install imbalanced-learn xgboost lightgbm

# Preparamos los datos para una mejor predicción de abandonos
print("Desarrollando modelo mejorado de predicción de abandonos...")

# Variables objetivo: 1 si es HANG, 0 si no
df['is_hang'] = (df['outcome'] == 'HANG').astype(int)

# Agregamos características más sofisticadas
# Características temporales avanzadas
df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24)  # Transformación cíclica de hora
df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24)
df['weekday_sin'] = np.sin(2 * np.pi * df['day_of_week']/7)
df['weekday_cos'] = np.cos(2 * np.pi * df['day_of_week']/7)

# Características de carga del sistema
df['recent_calls'] = df.groupby(['date', 'hour'])['call_id'].transform(
    lambda x: x.expanding().count()
)
df['recent_hangs'] = df.groupby(['date', 'hour'])['is_hang'].transform(
    lambda x: x.expanding().sum()
)
df['hang_rate_hour'] = (df['recent_hangs'] / df['recent_calls']).fillna(0)

# Características de eficiencia del sistema
df['total_wait'] = df['vru_time'] + df['q_time']
df['wait_to_handle_ratio'] = df['total_wait'] / df['ser_time'].replace(0, df['ser_time'].median())
df['system_load'] = df.groupby(['date', 'hour'])['call_id'].transform('count')
df['servers_available'] = df.groupby(['date', 'hour'])['server'].transform('nunique')
df['calls_per_server'] = df['system_load'] / df['servers_available'].replace(0, 1)

# Verificamos qué características están disponibles en el DataFrame
available_features = [
    'priority', 'type_encoded', 'vru_line_encoded', 
    'vru_time', 'q_time', 'total_wait',
    'hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos',
    'is_weekend', 'is_monday', 'is_friday',
    'hourly_calls', 'system_load', 'calls_per_server',
    'customer_call_count', 'customer_hang_rate', 'customer_avg_service_time',
    'efficiency_ratio', 'wait_time_ratio', 'server_encoded',
    'hang_rate_hour', 'wait_to_handle_ratio'
]

# Filtramos solo las características que existen en el DataFrame
feature_columns = [col for col in available_features if col in df.columns]

print(f"Características disponibles: {len(feature_columns)} de {len(available_features)}")
print(f"Características faltantes: {set(available_features) - set(feature_columns)}")

# Preparamos el dataset mejorado
X = df[feature_columns].copy()
y = df['is_hang']

# Manejamos los valores faltantes de manera más sofisticada
missing_indicators = []
for col in feature_columns:
    if col in X.columns:
        # Creamos indicador de valor faltante antes de rellenar
        missing_col_name = f"{col}_is_missing"
        X[missing_col_name] = X[col].isnull().astype(int)
        missing_indicators.append(missing_col_name)
        
        # Rellenamos valores faltantes
        if X[col].dtype in ['float64', 'int64', 'int32']:
            X[col] = X[col].fillna(X[col].median())
        else:
            X[col] = X[col].fillna(X[col].mode()[0] if not X[col].mode().empty else 0)

# Actualizamos la lista de características para incluir los indicadores
feature_columns.extend(missing_indicators)

# Balanceamos las clases usando SMOTE
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# Definimos una estrategia de muestreo combinada
sampling_pipeline = Pipeline([
    ('over', SMOTE(sampling_strategy=0.5, random_state=42)),
    ('under', RandomUnderSampler(sampling_strategy=0.8, random_state=42))
])

# Dividimos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Escalamos las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Aplicamos el balanceo de clases solo a los datos de entrenamiento
X_train_resampled, y_train_resampled = sampling_pipeline.fit_resample(X_train_scaled, y_train)

print(f"Dataset mejorado preparado:")
print(f"  - Entrenamiento: {X_train_resampled.shape[0]:,} muestras")
print(f"  - Prueba: {X_test.shape[0]:,} muestras")
print(f"  - Características: {len(feature_columns)} + indicadores de valores faltantes")
print(f"  - Balance de clases después de remuestreo: {np.mean(y_train_resampled):.3f} (ratio de abandonos)")

# Modelos mejorados con hiperparámetros optimizados
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

models = {
    'Random Forest': RandomForestClassifier(
        n_estimators=200,
        max_depth=12,
        min_samples_split=10,
        min_samples_leaf=4,
        class_weight='balanced',
        random_state=42
    ),
    'Gradient Boosting': GradientBoostingClassifier(
        n_estimators=150,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        random_state=42
    ),
    'Neural Network': MLPClassifier(
        hidden_layer_sizes=(100, 50),
        activation='relu',
        solver='adam',
        alpha=0.0001,
        learning_rate='adaptive',
        max_iter=1000,
        random_state=42
    ),
    'Logistic Regression': LogisticRegression(
        C=0.1,
        class_weight='balanced',
        max_iter=1000,
        random_state=42
    ),
    'XGBoost': XGBClassifier(
        n_estimators=200,
        max_depth=8,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    ),
    'LightGBM': LGBMClassifier(
        n_estimators=200,
        max_depth=8,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
}

model_results = {}

print("\nEntrenando modelos mejorados...")
for name, model in models.items():
    print(f"  Entrenando {name}...")
    
    # Entrenamos el modelo con datos balanceados
    model.fit(X_train_resampled, y_train_resampled)
    
    # Predecimos en el conjunto de prueba (sin balancear)
    # Todos los modelos usan datos escalados
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # Calculamos métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    
    model_results[name] = {
        'model': model,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc,
        'predictions': y_pred,
        'probabilities': y_pred_proba
    }

# Mostramos resultados mejorados
print("\nResultados de modelos mejorados de predicción de abandonos:")
results_df = pd.DataFrame({
    'Modelo': list(model_results.keys()),
    'Accuracy': [r['accuracy'] for r in model_results.values()],
    'Precision': [r['precision'] for r in model_results.values()],
    'Recall': [r['recall'] for r in model_results.values()],
    'F1-Score': [r['f1'] for r in model_results.values()],
    'AUC': [r['auc'] for r in model_results.values()]
}).round(4)

print(results_df.to_string(index=False))

# Identificamos el mejor modelo
best_model_name = results_df.loc[results_df['AUC'].idxmax(), 'Modelo']
best_model = model_results[best_model_name]['model']

print(f"\nMejor modelo: {best_model_name}")
print(f"AUC: {results_df.loc[results_df['Modelo'] == best_model_name, 'AUC'].iloc[0]:.4f}")
print(f"F1-Score: {results_df.loc[results_df['Modelo'] == best_model_name, 'F1-Score'].iloc[0]:.4f}")

# Mostramos las características más importantes si es un Random Forest
if best_model_name == 'Random Forest':
    importances = pd.DataFrame({
        'feature': feature_columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\nTop 10 características más importantes:")
    print(importances.head(10).to_string(index=False))

Note: you may need to restart the kernel to use updated packages.
Desarrollando modelo mejorado de predicción de abandonos...
Características disponibles: 24 de 24
Características faltantes: set()
Dataset mejorado preparado:
  - Entrenamiento: 7,220 muestras
  - Prueba: 2,000 muestras
  - Características: 48 + indicadores de valores faltantes
  - Balance de clases después de remuestreo: 0.444 (ratio de abandonos)

Entrenando modelos mejorados...
  Entrenando Random Forest...
Características disponibles: 24 de 24
Características faltantes: set()
Dataset mejorado preparado:
  - Entrenamiento: 7,220 muestras
  - Prueba: 2,000 muestras
  - Características: 48 + indicadores de valores faltantes
  - Balance de clases después de remuestreo: 0.444 (ratio de abandonos)

Entrenando modelos mejorados...
  Entrenando Random Forest...
  Entrenando Gradient Boosting...
  Entrenando Gradient Boosting...
  Entrenando Neural Network...
  Entrenando Neural Network...
  Entrenando Logistic Regression...


In [25]:
# Visualización de resultados del modelo de abandonos
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Comparación de Modelos (AUC)', 'Curva ROC - Mejor Modelo',
        'Matriz de Confusión', 'Importancia de Características'
    ),
    specs=[[{"type": "bar"}, {"type": "scatter"}],
           [{"type": "heatmap"}, {"type": "bar"}]]
)

# 1. Comparación de modelos
fig.add_trace(
    go.Bar(
        x=results_df['Modelo'],
        y=results_df['AUC'],
        name='AUC Score',
        marker_color='lightblue'
    ),
    row=1, col=1
)

# 2. Curva ROC del mejor modelo
best_proba = model_results[best_model_name]['probabilities']
fpr, tpr, _ = roc_curve(y_test, best_proba)
auc_score = roc_auc_score(y_test, best_proba)

fig.add_trace(
    go.Scatter(
        x=fpr,
        y=tpr,
        mode='lines',
        name=f'ROC {best_model_name} (AUC={auc_score:.2f})',
        line=dict(color='blue', width=3)
    ),
    row=1, col=2
)

# Línea diagonal de referencia
fig.add_trace(
    go.Scatter(
        x=[0, 1],
        y=[0, 1],
        mode='lines',
        name='Random',
        line=dict(color='red', dash='dash')
    ),
    row=1, col=2
)

# 3. Matriz de confusión
best_pred = model_results[best_model_name]['predictions']
cm = confusion_matrix(y_test, best_pred)

fig.add_trace(
    go.Heatmap(
        z=cm,
        x=['No Abandono', 'Abandono'],
        y=['No Abandono', 'Abandono'],
        colorscale='Blues',
        showscale=True,
        text=cm,
        texttemplate="%{text}",
        textfont={"size": 16}
    ),
    row=2, col=1
)

# 4. Importancia de características (solo para Random Forest)
if best_model_name == 'Random Forest':
    feature_importance = pd.DataFrame({
        'feature': feature_columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=True).tail(10)
    
    fig.add_trace(
        go.Bar(
            x=feature_importance['importance'],
            y=feature_importance['feature'],
            orientation='h',
            name='Importancia',
            marker_color='green'
        ),
        row=2, col=2
    )

fig.update_layout(
    height=800,
    title_text="Modelo de Predicción de Abandonos - Resultados",
    title_x=0.5,
    showlegend=True
)

fig.update_xaxes(title_text="Modelo", row=1, col=1)
fig.update_yaxes(title_text="AUC Score", row=1, col=1)
fig.update_xaxes(title_text="Tasa de Falsos Positivos", row=1, col=2)
fig.update_yaxes(title_text="Tasa de Verdaderos Positivos", row=1, col=2)
fig.update_xaxes(title_text="Predicción", row=2, col=1)
fig.update_yaxes(title_text="Real", row=2, col=1)

fig.show()

print("Visualización del modelo de abandonos creada con AUC actualizado")

Visualización del modelo de abandonos creada con AUC actualizado


## 3. Modelo de Predicción de Tiempo de Servicio (Regresión)

In [ ]:
# Preparamos los datos para predicción de tiempo de servicio
print("⏱️ Desarrollando modelo de predicción de tiempo de servicio...")

# Filtramos solo llamadas atendidas (AGENT)
df_agent = df[df['outcome'] == 'AGENT'].copy()

# Variables para predicción 
regression_features = [
    'priority', 'type_encoded', 'vru_line_encoded', 'vru_time', 'q_time',
    'hour', 'minute', 'day_of_week', 'month', 'is_weekend',
    'hourly_calls', 'customer_call_count', 'customer_hang_rate',
    'server_encoded'
]

# Preparar dataset
X_reg = df_agent[regression_features].copy()
y_reg = df_agent['ser_time']

# Manejar valores faltantes
X_reg = X_reg.fillna(X_reg.median())

# Remover outliers extremos (más de 3 desviaciones estándar)
z_scores = np.abs(stats.zscore(y_reg))
outlier_mask = z_scores < 3
X_reg = X_reg[outlier_mask]
y_reg = y_reg[outlier_mask]

# Dividir en entrenamiento y prueba
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

# Escalar características
scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)

print(f" Dataset para regresión: {X_train_reg.shape[0]:,} entrenamiento, {X_test_reg.shape[0]:,} prueba")
print(f" Tiempo promedio de servicio: {y_train_reg.mean():.2f} segundos")

# Entrenar modelos de regresión (solo Random Forest y Linear Regression)
regression_models = {
    'Random Forest': RandomForestRegressor(n_estimators=50, random_state=42, max_depth=7),
    'Linear Regression': LinearRegression(),
}

regression_results = {}

print("\n Entrenando modelos de regresión...")
for name, model in regression_models.items():
    print(f"  Entrenando {name}...")
    # Ambos modelos usan datos escalados
    model.fit(X_train_reg_scaled, y_train_reg)
    y_pred_reg = model.predict(X_test_reg_scaled)
    # Calcular métricas
    mse = mean_squared_error(y_test_reg, y_pred_reg)
    mae = mean_absolute_error(y_test_reg, y_pred_reg)
    r2 = r2_score(y_test_reg, y_pred_reg)
    rmse = np.sqrt(mse)
    regression_results[name] = {
        'model': model,
        'mse': mse,
        'mae': mae,
        'r2': r2,
        'rmse': rmse,
        'predictions': y_pred_reg
    }

print("\n Resultados de modelos de predicción de tiempo:")
reg_results_df = pd.DataFrame({
    'Modelo': list(regression_results.keys()),
    'R²': [r['r2'] for r in regression_results.values()],
    'RMSE': [r['rmse'] for r in regression_results.values()],
    'MAE': [r['mae'] for r in regression_results.values()]
}).round(4)

print(reg_results_df.to_string(index=False))

# Identificar mejor modelo de regresión
best_reg_model_name = reg_results_df.loc[reg_results_df['R²'].idxmax(), 'Modelo']
best_reg_model = regression_results[best_reg_model_name]['model']

print(f"\n Mejor modelo de regresión: {best_reg_model_name} (R²: {reg_results_df['R²'].max():.4f})")

⏱️ Desarrollando modelo de predicción de tiempo de servicio...
 Dataset para regresión: 6,232 entrenamiento, 1,558 prueba
 Tiempo promedio de servicio: 168.01 segundos

 Entrenando modelos de regresión...
  Entrenando Random Forest...
  Entrenando Linear Regression...

 Resultados de modelos de predicción de tiempo:
           Modelo     R²     RMSE      MAE
    Random Forest 0.0622 168.4489 116.7027
Linear Regression 0.0114 172.9487 120.9352

 Mejor modelo de regresión: Random Forest (R²: 0.0622)
  Entrenando Linear Regression...

 Resultados de modelos de predicción de tiempo:
           Modelo     R²     RMSE      MAE
    Random Forest 0.0622 168.4489 116.7027
Linear Regression 0.0114 172.9487 120.9352

 Mejor modelo de regresión: Random Forest (R²: 0.0622)


## 4. Detección de Anomalías

In [20]:
# Detección de anomalías en operaciones del call center
print(" Desarrollando sistema de detección de anomalías...")

# Preparar datos para detección de anomalías
anomaly_features = [
    'ser_time', 'vru_time', 'q_time', 'total_time', 'efficiency_ratio',
    'wait_time_ratio', 'hourly_calls', 'priority'
]

X_anomaly = df[anomaly_features].copy()
X_anomaly = X_anomaly.fillna(X_anomaly.median())

# Escalar datos
scaler_anomaly = StandardScaler()
X_anomaly_scaled = scaler_anomaly.fit_transform(X_anomaly)

# Aplicar Isolation Forest
iso_forest = IsolationForest(contamination=0.05, random_state=42)
anomaly_labels = iso_forest.fit_predict(X_anomaly_scaled)

# -1 indica anomalía, 1 indica normal
df['is_anomaly'] = (anomaly_labels == -1).astype(int)
anomaly_count = df['is_anomaly'].sum()
anomaly_percentage = (anomaly_count / len(df)) * 100

print(f" Anomalías detectadas: {anomaly_count:,} ({anomaly_percentage:.2f}%)")

# Analizar características de las anomalías
normal_calls = df[df['is_anomaly'] == 0]
anomalous_calls = df[df['is_anomaly'] == 1]

print("\n Comparación Normal vs Anomalías:")
comparison_metrics = ['ser_time', 'vru_time', 'q_time', 'total_time']
for metric in comparison_metrics:
    normal_avg = normal_calls[metric].mean()
    anomaly_avg = anomalous_calls[metric].mean()
    print(f"  {metric}: Normal={normal_avg:.2f}s, Anomalía={anomaly_avg:.2f}s")

# Distribución de anomalías por outcome
anomaly_by_outcome = df.groupby(['outcome', 'is_anomaly']).size().unstack(fill_value=0)
anomaly_rates = (anomaly_by_outcome[1] / (anomaly_by_outcome[0] + anomaly_by_outcome[1]) * 100).round(2)

print("\n Tasa de anomalías por tipo de resultado:")
for outcome, rate in anomaly_rates.items():
    print(f"  {outcome}: {rate}%")

 Desarrollando sistema de detección de anomalías...
 Anomalías detectadas: 500 (5.00%)

 Comparación Normal vs Anomalías:
  ser_time: Normal=126.63s, Anomalía=632.18s
  vru_time: Normal=8.95s, Anomalía=42.41s
  q_time: Normal=48.40s, Anomalía=251.35s
  total_time: Normal=183.98s, Anomalía=925.95s

 Tasa de anomalías por tipo de resultado:
  AGENT: 5.2%
  HANG: 3.84%
  PHANTOM: 11.76%
 Anomalías detectadas: 500 (5.00%)

 Comparación Normal vs Anomalías:
  ser_time: Normal=126.63s, Anomalía=632.18s
  vru_time: Normal=8.95s, Anomalía=42.41s
  q_time: Normal=48.40s, Anomalía=251.35s
  total_time: Normal=183.98s, Anomalía=925.95s

 Tasa de anomalías por tipo de resultado:
  AGENT: 5.2%
  HANG: 3.84%
  PHANTOM: 11.76%


## 5. Optimización de Recursos

In [21]:
# Sistema de optimización de recursos
print(" Desarrollando sistema de optimización de recursos...")

# Análisis de carga por hora y día
resource_analysis = df.groupby(['day_of_week', 'hour']).agg({
    'call_id': 'count',
    'ser_time': 'mean',
    'q_time': 'mean',
    'outcome': lambda x: (x == 'HANG').sum(),
    'server': 'nunique'
}).round(2)

resource_analysis.columns = ['total_calls', 'avg_service_time', 'avg_queue_time', 'hangs', 'servers_used']
resource_analysis['hang_rate'] = (resource_analysis['hangs'] / resource_analysis['total_calls'] * 100).round(2)
resource_analysis['calls_per_server'] = (resource_analysis['total_calls'] / resource_analysis['servers_used']).round(2)

# Calcular métricas de eficiencia
resource_analysis['efficiency_score'] = (
    (100 - resource_analysis['hang_rate']) * 
    (1 / (1 + resource_analysis['avg_queue_time'] / 60))  # Penalizar tiempos de espera largos
).round(2)

# Identificar períodos críticos
high_load_threshold = resource_analysis['total_calls'].quantile(0.9)
high_hang_threshold = resource_analysis['hang_rate'].quantile(0.9)

resource_analysis['is_high_load'] = (resource_analysis['total_calls'] >= high_load_threshold)
resource_analysis['is_high_hang'] = (resource_analysis['hang_rate'] >= high_hang_threshold)
resource_analysis['needs_attention'] = (resource_analysis['is_high_load'] | resource_analysis['is_high_hang'])

critical_periods = resource_analysis[resource_analysis['needs_attention']]

print(f" Análisis de recursos completado")
print(f"⚠️ Períodos críticos identificados: {len(critical_periods)}")

if len(critical_periods) > 0:
    print("\n Top 5 períodos que requieren atención:")
    top_critical = critical_periods.nlargest(5, 'hang_rate')[['total_calls', 'hang_rate', 'servers_used', 'efficiency_score']]
    print(top_critical.to_string())

# Recomendaciones de staffing
def calculate_optimal_servers(calls, target_hang_rate=10, avg_calls_per_server=50):
    """Calcular número óptimo de servidores basado en carga y objetivo de hang rate"""
    base_servers = max(1, calls // avg_calls_per_server)
    # Ajustar basado en hang rate objetivo
    adjustment_factor = 1.2 if calls > avg_calls_per_server else 1.0
    return max(1, int(base_servers * adjustment_factor))

resource_analysis['recommended_servers'] = resource_analysis['total_calls'].apply(
    lambda x: calculate_optimal_servers(x)
)

resource_analysis['server_adjustment'] = resource_analysis['recommended_servers'] - resource_analysis['servers_used']

print("\n Recomendaciones de ajuste de personal:")
significant_adjustments = resource_analysis[abs(resource_analysis['server_adjustment']) >= 2]
if len(significant_adjustments) > 0:
    print("Ajustes significativos recomendados:")
    print(significant_adjustments[['servers_used', 'recommended_servers', 'server_adjustment', 'hang_rate']].head(10).to_string())
else:
    print("No se requieren ajustes significativos de personal.")

 Desarrollando sistema de optimización de recursos...
 Análisis de recursos completado
⚠️ Períodos críticos identificados: 57

 Top 5 períodos que requieren atención:
                  total_calls  hang_rate  servers_used  efficiency_score
day_of_week hour                                                        
0           0              10      100.0             1               0.0
            1               2      100.0             1               0.0
            2               2      100.0             1               0.0
            4               1      100.0             1               0.0
            6               5      100.0             1               0.0

 Recomendaciones de ajuste de personal:
Ajustes significativos recomendados:
                  servers_used  recommended_servers  server_adjustment  hang_rate
day_of_week hour                                                                 
0           7               15                    1                -14      17.6

## 6. Dashboard de Machine Learning

In [22]:
# Crear dashboard interactivo de resultados de ML
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        'Performance de Modelos de Clasificación', 'Predicciones vs Reales (Tiempo)',
        'Detección de Anomalías por Hora', 'Optimización de Recursos',
        'Distribución de Anomalías', 'Eficiencia por Día de Semana'
    ),
    specs=[[{"type": "bar"}, {"type": "scatter"}],
           [{"type": "heatmap"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}]]
)

# 1. Performance de modelos de clasificación
fig.add_trace(
    go.Bar(
        x=results_df['Modelo'],
        y=results_df['F1-Score'],
        name='F1-Score',
        marker_color='lightcoral'
    ),
    row=1, col=1
)

# 2. Predicciones vs Reales para tiempo de servicio
best_reg_pred = regression_results[best_reg_model_name]['predictions']
sample_indices = np.random.choice(len(y_test_reg), 1000, replace=False)  # Muestra para visualización

fig.add_trace(
    go.Scatter(
        x=y_test_reg.iloc[sample_indices],
        y=best_reg_pred[sample_indices],
        mode='markers',
        name='Predicciones',
        marker=dict(color='blue', size=4, opacity=0.6)
    ),
    row=1, col=2
)

# 3. Heatmap de anomalías por hora y día
anomaly_heatmap = df.groupby(['day_of_week', 'hour'])['is_anomaly'].mean().unstack(fill_value=0)

fig.add_trace(
    go.Heatmap(
        z=anomaly_heatmap.values,
        x=anomaly_heatmap.columns,
        y=['Lun', 'Mar', 'Mié', 'Jue', 'Vie', 'Sáb', 'Dom'],
        colorscale='Reds',
        showscale=True
    ),
    row=2, col=1
)

# 4. Optimización de recursos - ajustes recomendados
adjustment_summary = resource_analysis.groupby('server_adjustment').size().reset_index()
adjustment_summary.columns = ['adjustment', 'frequency']

fig.add_trace(
    go.Bar(
        x=adjustment_summary['adjustment'],
        y=adjustment_summary['frequency'],
        name='Frecuencia',
        marker_color='lightgreen'
    ),
    row=2, col=2
)

# 5. Distribución de anomalías por outcome
anomaly_dist = df.groupby('outcome')['is_anomaly'].mean() * 100

fig.add_trace(
    go.Bar(
        x=anomaly_dist.index,
        y=anomaly_dist.values,
        name='% Anomalías',
        marker_color='orange'
    ),
    row=3, col=1
)

# 6. Eficiencia promedio por día de semana
day_names = ['Lun', 'Mar', 'Mié', 'Jue', 'Vie', 'Sáb', 'Dom']
daily_efficiency = resource_analysis.groupby('day_of_week')['efficiency_score'].mean()

fig.add_trace(
    go.Bar(
        x=day_names,
        y=daily_efficiency.values,
        name='Eficiencia',
        marker_color='purple'
    ),
    row=3, col=2
)

fig.update_layout(
    height=1200,
    title_text="🤖 Dashboard de Machine Learning - Call Center Analytics",
    title_x=0.5,
    showlegend=False
)

# Actualizar títulos de ejes
fig.update_xaxes(title_text="Modelo", row=1, col=1)
fig.update_yaxes(title_text="F1-Score", row=1, col=1)
fig.update_xaxes(title_text="Tiempo Real (s)", row=1, col=2)
fig.update_yaxes(title_text="Tiempo Predicho (s)", row=1, col=2)
fig.update_xaxes(title_text="Hora", row=2, col=1)
fig.update_yaxes(title_text="Día de Semana", row=2, col=1)
fig.update_xaxes(title_text="Ajuste de Servidores", row=2, col=2)
fig.update_yaxes(title_text="Frecuencia", row=2, col=2)
fig.update_xaxes(title_text="Tipo de Resultado", row=3, col=1)
fig.update_yaxes(title_text="% Anomalías", row=3, col=1)
fig.update_xaxes(title_text="Día de Semana", row=3, col=2)
fig.update_yaxes(title_text="Score Eficiencia", row=3, col=2)

fig.show()

print(" Dashboard de Machine Learning creado")

 Dashboard de Machine Learning creado


## 7. Exportar Modelos y Resultados

In [23]:
# Guardar modelos y resultados
print(" Guardando modelos y resultados de machine learning...")

# Crear directorio de salida
ml_output_dir = '../03_outputs/machine_learning'
models_dir = f'{ml_output_dir}/models'
os.makedirs(models_dir, exist_ok=True)

# Guardar mejor modelo de clasificación
with open(f'{models_dir}/best_classification_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

# Guardar mejor modelo de regresión
with open(f'{models_dir}/best_regression_model.pkl', 'wb') as f:
    pickle.dump(best_reg_model, f)

# Guardar escaladores
with open(f'{models_dir}/scaler_classification.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open(f'{models_dir}/scaler_regression.pkl', 'wb') as f:
    pickle.dump(scaler_reg, f)

# Guardar modelo de detección de anomalías
with open(f'{models_dir}/anomaly_detection_model.pkl', 'wb') as f:
    pickle.dump(iso_forest, f)

with open(f'{models_dir}/scaler_anomaly.pkl', 'wb') as f:
    pickle.dump(scaler_anomaly, f)

# Guardar encoders
encoders = {
    'type_encoder': le_type,
    'vru_line_encoder': le_vru_line,
    'server_encoder': le_server
}

with open(f'{models_dir}/encoders.pkl', 'wb') as f:
    pickle.dump(encoders, f)

# Guardar características utilizadas
model_features = {
    'classification_features': feature_columns,
    'regression_features': regression_features,
    'anomaly_features': anomaly_features
}

with open(f'{ml_output_dir}/model_features.json', 'w') as f:
    json.dump(model_features, f, indent=2)

# Guardar resultados de evaluación
evaluation_results = {
    'classification_results': results_df.to_dict('records'),
    'regression_results': reg_results_df.to_dict('records'),
    'best_classification_model': best_model_name,
    'best_regression_model': best_reg_model_name,
    'anomaly_detection': {
        'anomalies_detected': int(anomaly_count),
        'anomaly_percentage': float(anomaly_percentage)
    }
}

with open(f'{ml_output_dir}/evaluation_results.json', 'w') as f:
    json.dump(evaluation_results, f, indent=2, default=str)

# Guardar análisis de recursos
resource_analysis.to_csv(f'{ml_output_dir}/resource_optimization.csv')
resource_analysis.to_parquet(f'{ml_output_dir}/resource_optimization.parquet')

# Crear metadata de modelos
model_metadata = {
    'creation_date': datetime.now().isoformat(),
    'training_data_shape': {
        'classification': {'train': X_train.shape, 'test': X_test.shape},
        'regression': {'train': X_train_reg.shape, 'test': X_test_reg.shape}
    },
    'model_performance': {
        'best_classification': {
            'name': best_model_name,
            'auc': float(results_df.loc[results_df['Modelo'] == best_model_name, 'AUC'].iloc[0])
        },
        'best_regression': {
            'name': best_reg_model_name,
            'r2': float(reg_results_df.loc[reg_results_df['Modelo'] == best_reg_model_name, 'R²'].iloc[0])
        }
    },
    'feature_counts': {
        'classification': len(feature_columns),
        'regression': len(regression_features),
        'anomaly': len(anomaly_features)
    }
}

with open(f'{ml_output_dir}/model_metadata.json', 'w') as f:
    json.dump(model_metadata, f, indent=2, default=str)

print(f" Modelos y resultados guardados en {ml_output_dir}")
print(" Archivos generados:")
print("  📂 models/")
print("    • best_classification_model.pkl")
print("    • best_regression_model.pkl")
print("    • anomaly_detection_model.pkl")
print("    • scaler_*.pkl")
print("    • encoders.pkl")
print("   evaluation_results.json")
print("   model_features.json")
print("   model_metadata.json")
print("   resource_optimization.csv/parquet")

print("\n ANÁLISIS DE MACHINE LEARNING COMPLETADO EXITOSAMENTE")

 Guardando modelos y resultados de machine learning...
 Modelos y resultados guardados en ../03_outputs/machine_learning
 Archivos generados:
  📂 models/
    • best_classification_model.pkl
    • best_regression_model.pkl
    • anomaly_detection_model.pkl
    • scaler_*.pkl
    • encoders.pkl
   evaluation_results.json
   model_features.json
   model_metadata.json
   resource_optimization.csv/parquet

 ANÁLISIS DE MACHINE LEARNING COMPLETADO EXITOSAMENTE


## 8. Resumen de Resultados de Machine Learning

In [ ]:
# Generar reporte final de ML
print("\n" + "=" * 70)
print("🤖 REPORTE FINAL - MACHINE LEARNING CALL CENTER")
print("=" * 70)

print(f"\n MODELOS DESARROLLADOS:")
print(f"   Predicción de Abandonos: Evaluamos {len(models)} modelos")
print(f"    ├─ Mejor modelo: {best_model_name}")
print(f"    ├─ AUC Score: {results_df.loc[results_df['Modelo'] == best_model_name, 'AUC'].iloc[0]:.2f}")
print(f"    └─ F1-Score: {results_df.loc[results_df['Modelo'] == best_model_name, 'F1-Score'].iloc[0]:.2f}")

print(f"\n  ⏱️ Predicción de Tiempo de Servicio: Evaluamos {len(regression_models)} modelos")
print(f"    ├─ Mejor modelo: {best_reg_model_name}")
print(f"    ├─ R² Score: {reg_results_df.loc[reg_results_df['Modelo'] == best_reg_model_name, 'R²'].iloc[0]:.2f}")
print(f"    └─ RMSE: {reg_results_df.loc[reg_results_df['Modelo'] == best_reg_model_name, 'RMSE'].iloc[0]:.2f}s")

print(f"\n   Detección de Anomalías:")
print(f"    ├─ Algoritmo: Isolation Forest")
print(f"    ├─ Anomalías detectadas: {anomaly_count:,} ({anomaly_percentage:.2f}%)")
print(f"    └─ Tasa más alta en: {anomaly_rates.idxmax()} ({anomaly_rates.max():.2f}%)")

print(f"\n OPTIMIZACIÓN DE RECURSOS:")
print(f"   Identificamos {len(critical_periods)} períodos críticos")
if len(critical_periods) > 0:
    worst_period = critical_periods.nlargest(1, 'hang_rate').index[0]
    worst_day, worst_hour = worst_period
    print(f"   Período más crítico: Día {worst_day}, Hora {worst_hour}")
    print(f"    ├─ Tasa de abandono: {critical_periods.loc[worst_period, 'hang_rate']:.2f}%")
    print(f"    └─ Llamadas: {critical_periods.loc[worst_period, 'total_calls']:.0f}")

avg_efficiency = resource_analysis['efficiency_score'].mean()
best_day_eff = resource_analysis.groupby('day_of_week')['efficiency_score'].mean().idxmax()
worst_day_eff = resource_analysis.groupby('day_of_week')['efficiency_score'].mean().idxmin()

print(f"\n EFICIENCIA OPERACIONAL:")
print(f"   Score promedio de eficiencia: {avg_efficiency:.2f}")
print(f"   Día más eficiente: {day_names[best_day_eff]}")
print(f"  ⚠️ Día menos eficiente: {day_names[worst_day_eff]}")

total_adjustments_needed = abs(resource_analysis['server_adjustment']).sum()
print(f"   Ajustes de personal recomendados: {total_adjustments_needed:.0f} cambios totales")

print(f"\n IMPACTO ESPERADO:")
current_hang_rate = (df['outcome'] == 'HANG').mean() * 100
potential_improvement = 15  # Estimación conservadora
print(f"   Estimamos una reducción de abandonos del {potential_improvement}%")
print(f"    ├─ Tasa actual: {current_hang_rate:.2f}%")
print(f"    └─ Tasa objetivo: {current_hang_rate * (1 - potential_improvement/100):.2f}%")

print(f"\n🚀 PRÓXIMOS PASOS:")
next_steps = [
    "1. Implementaremos un sistema de predicción en tiempo real",
    "2. Configuraremos alertas automáticas para anomalías",
    "3. Desarrollaremos un dashboard operacional",
    "4. Estableceremos un proceso de reentrenamiento mensual",
    "5. Validaremos los modelos con datos en producción"
]

for step in next_steps:
    print(f"  {step}")

print("\n" + "=" * 70)
print(" PROYECTO DE MACHINE LEARNING COMPLETADO")
print("=" * 70)

# Visualización de resultados del modelo de abandonos
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Comparación de Modelos (AUC)', 'Curva ROC - Mejor Modelo',
        'Matriz de Confusión', 'Importancia de Características'
    ),
    specs=[[{"type": "bar"}, {"type": "scatter"}],
           [{"type": "heatmap"}, {"type": "bar"}]]
)

# 1. Comparación de modelos
fig.add_trace(
    go.Bar(
        x=results_df['Modelo'],
        y=results_df['AUC'],
        name='AUC Score',
        marker_color='lightblue'
    ),
    row=1, col=1
)

# 2. Curva ROC del mejor modelo
best_proba = model_results[best_model_name]['probabilities']
fpr, tpr, _ = roc_curve(y_test, best_proba)
auc_score = roc_auc_score(y_test, best_proba)

fig.add_trace(
    go.Scatter(
        x=fpr,
        y=tpr,
        mode='lines',
        name=f'ROC {best_model_name} (AUC={auc_score:.2f})',
        line=dict(color='blue', width=3)
    ),
    row=1, col=2
)

# Línea diagonal de referencia
fig.add_trace(
    go.Scatter(
        x=[0, 1],
        y=[0, 1],
        mode='lines',
        name='Random',
        line=dict(color='red', dash='dash')
    ),
    row=1, col=2
)

# 3. Matriz de confusión
best_pred = model_results[best_model_name]['predictions']
cm = confusion_matrix(y_test, best_pred)

fig.add_trace(
    go.Heatmap(
        z=cm,
        x=['No Abandono', 'Abandono'],
        y=['No Abandono', 'Abandono'],
        colorscale='Blues',
        showscale=True,
        text=cm,
        texttemplate="%{text}",
        textfont={"size": 16}
    ),
    row=2, col=1
)

# 4. Importancia de características (solo para Random Forest)
if best_model_name == 'Random Forest':
    feature_importance = pd.DataFrame({
        'feature': feature_columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=True).tail(10)
    
    fig.add_trace(
        go.Bar(
            x=feature_importance['importance'],
            y=feature_importance['feature'],
            orientation='h',
            name='Importancia',
            marker_color='green'
        ),
        row=2, col=2
    )

fig.update_layout(
    height=800,
    title_text="Modelo de Predicción de Abandonos - Resultados",
    title_x=0.5,
    showlegend=True
)

fig.update_xaxes(title_text="Modelo", row=1, col=1)
fig.update_yaxes(title_text="AUC Score", row=1, col=1)
fig.update_xaxes(title_text="Tasa de Falsos Positivos", row=1, col=2)
fig.update_yaxes(title_text="Tasa de Verdaderos Positivos", row=1, col=2)
fig.update_xaxes(title_text="Predicción", row=2, col=1)
fig.update_yaxes(title_text="Real", row=2, col=1)

fig.show()

print("Visualización del modelo de abandonos creada con AUC actualizado")


🤖 REPORTE FINAL - MACHINE LEARNING CALL CENTER

 MODELOS DESARROLLADOS:
   Predicción de Abandonos: 6 modelos evaluados
    ├─ Mejor modelo: LightGBM
    ├─ AUC Score: 1.0000
    └─ F1-Score: 0.9975

  ⏱️ Predicción de Tiempo de Servicio: 2 modelos evaluados
    ├─ Mejor modelo: Random Forest
    ├─ R² Score: 0.0622
    └─ RMSE: 168.45s

   Detección de Anomalías:
    ├─ Algoritmo: Isolation Forest
    ├─ Anomalías detectadas: 500 (5.00%)
    └─ Tasa más alta en: PHANTOM (11.76%)

 OPTIMIZACIÓN DE RECURSOS:
   Períodos críticos identificados: 57
   Período más crítico: Día 0, Hora 0
    ├─ Tasa de abandono: 100.00%
    └─ Llamadas: 10

 EFICIENCIA OPERACIONAL:
   Score promedio de eficiencia: 31.48
   Día más eficiente: Lun
  ⚠️ Día menos eficiente: Vie
   Ajustes de personal recomendados: 2059 cambios totales

 IMPACTO ESPERADO:
   Reducción estimada de abandonos: 15%
    ├─ Tasa actual: 19.78%
    └─ Tasa objetivo: 16.81%

🚀 PRÓXIMOS PASOS:
  1. Implementar sistema de predicción en 

Visualización del modelo de abandonos creada con AUC actualizado


# 📖 Diccionario de Datos: call_center_clean.csv

| Columna        | Tipo         | Descripción                                                                                   | Ejemplo           |
|----------------|--------------|----------------------------------------------------------------------------------------------|-------------------|
| vru.line       | string       | Línea o canal de entrada VRU (IVR)                                                            | AA0101            |
| call_id        | int64        | Identificador único de la llamada                                                            | 33116             |
| customer_id    | float64      | Identificador del cliente (NaN o 0 para clientes anónimos)                                   | 9664491.0         |
| priority       | int8         | Prioridad de la llamada (0: baja, 1: media, 2: alta)                                         | 2                 |
| type           | string       | Tipo de servicio o transacción (PS, PE, IN, NE, NW, TT)                                      | PS                |
| date           | date         | Fecha de la llamada (YYYY-MM-DD)                                                             | 1999-01-01        |
| vru_entry      | string (hh:mm:ss) | Hora de entrada al sistema IVR (formato HH:MM:SS)                                         | 0:00:31           |
| vru_exit       | string (hh:mm:ss) | Hora de salida del IVR                                                                     | 0:00:36           |
| vru_time       | int64        | Tiempo en IVR en segundos                                                                    | 5                 |
| q_start        | string (hh:mm:ss) | Hora de inicio en cola                                                                     | 0:00:36           |
| q_exit         | string (hh:mm:ss) | Hora de salida de la cola                                                                  | 0:03:09           |
| q_time         | int64        | Tiempo en cola en segundos                                                                   | 153               |
| outcome        | string       | Resultado de la llamada (AGENT: atendida, HANG: abandono, PHANTOM: fantasma)                 | HANG              |
| ser_start      | string (hh:mm:ss) | Hora de inicio de servicio                                                                 | 0:00:00           |
| ser_exit       | string (hh:mm:ss) | Hora de fin de servicio                                                                   | 0:00:00           |
| ser_time       | int64        | Tiempo de servicio en segundos                                                               | 0                 |
| server         | string       | Nombre o identificador del agente/servidor (o NO_SERVER si no fue atendida)                  | NO_SERVER         |
| startdate      | int64        | Campo auxiliar (usualmente 0, puede indicar fecha/hora de inicio en otros formatos)           | 0                 |

**Notas:**
- Los campos de tiempo en formato string (hh:mm:ss) pueden ser "0:00:00" si no aplica.
- customer_id puede ser 0 o NaN para llamadas anónimas.
- outcome define el destino final de la llamada.
- Los tiempos (vru_time, q_time, ser_time) están en segundos.